<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#!pip install onnx onnxruntime


In [2]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name_condition = "condition"
input_name_X = "X"
input_name_Y = "Y"
where_output_name = "WhereOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensors
#-------------------------------------------------------------------------------

# Create 2-rank input tensors
input_tensor_condition = onnx.helper.make_tensor_value_info(
    input_name_condition, onnx.TensorProto.BOOL, [None, None]
)

input_tensor_X = onnx.helper.make_tensor_value_info(
    input_name_X, onnx.TensorProto.FLOAT, [None, None]
)

input_tensor_Y = onnx.helper.make_tensor_value_info(
    input_name_Y, onnx.TensorProto.FLOAT, [None, None]
)

# Create output tensor
where_output = onnx.helper.make_tensor_value_info(
    where_output_name, onnx.TensorProto.FLOAT, [None, None]
)

# Define Where node: Z = Where(condition, X, Y)
where_node = onnx.helper.make_node(
    "Where",
    [input_name_condition, input_name_X, input_name_Y],
    [where_output_name]
)

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[where_node],
    name="Where",
    inputs=[input_tensor_condition, input_tensor_X, input_tensor_Y],
    outputs=[where_output]
)

# Create the ONNX model (Where available in opset 16)
model = onnx.helper.make_model(
    graph,
    opset_imports=[onnx.helper.make_opsetid("", 16)]
)

onnx.checker.check_model(model)

# Save the model
onnx.save(model, "where.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("where.onnx")

def do_where(cond, x, y):
    output = session.run(
        None,
        {
            input_name_condition: cond,
            input_name_X: x,
            input_name_Y: y,
        },
    )

    c_f = np.array2string(cond, separator=',', max_line_width=np.inf).replace('\n', '')
    x_f = np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '')
    y_f = np.array2string(y, separator=',', max_line_width=np.inf).replace('\n', '')
    o_f = np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '')

    print(f"condition={c_f}")
    print(f"X={x_f}")
    print(f"Y={y_f}")
    print(f"Result: Where(condition,X,Y)={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')

#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Example 1 (real/float section)

print("\n## Example 1")

condition_1 = np.array([[True, False, True]], dtype=bool)

x_1 = np.array([[9.0, 8.0, 7.1]], dtype=np.float32)
y_1 = np.array([[6.0, 5.0, 4.0]], dtype=np.float32)

do_where(condition_1, x_1, y_1)


# Example 2 (real section)

print("\n## Example 2")

condition_2 = np.array([
    [True,  True],
    [True,  False],
    [False, True]
], dtype=bool)

x_2 = np.array([
    [1.0, 2.0],
    [3.0, 4.0],
    [5.0, 6.0]
], dtype=np.float32)

y_2 = np.array([
    [12.0, 11.0],
    [10.0, 9.0],
    [8.0, 7.0]
], dtype=np.float32)

do_where(condition_2, x_2, y_2)





#--------------------------------------------------------------------
# Floating-point special values 
#--------------------------------------------------------------------

print("\n## Example 3a (IEEE-754)")

condition_3 = np.array([[True, False, True]], dtype=bool)

x_3 = np.array([[19.0, 28.0, 37.1]], dtype=np.float32)

y_3 = np.array([[16.0, 25.0, 34.0]], dtype=np.float32)

do_where(condition_3, x_3, y_3) 

#--------------------------------------------------------------------
# Floating-point special values (+0, -0, +inf, -inf, nan)
#--------------------------------------------------------------------

print("\n## Example 3b (IEEE-754 special values)")

condition_3 = np.array([[True, False, True, False, True]], dtype=bool)

x_3 = np.array([[+0.0, +0.0, float("inf"), float("inf"), float("nan")]], dtype=np.float32)

y_3 = np.array([[+0.0, -0.0, float("-inf"), float("-inf"), 1.0]], dtype=np.float32)

do_where(condition_3, x_3, y_3)


#--------------------------------------------------------------------
# Integer example
#--------------------------------------------------------------------

print("\n## Example 4 (integer)")

condition_4 = np.array([
    [True,  True],
    [True,  False],
    [False, True]
], dtype=bool)

x_4 = np.array([
    [1, 20],
    [3, 40],
    [5, 60]
], dtype=np.int32)

y_4 = np.array([
    [12, 110],
    [10, 90],
    [8, 70]
], dtype=np.int32)

do_where(condition_4, x_4.astype(np.float32), y_4.astype(np.float32))


## Example 1
condition=[[ True,False, True]]
X=[[9.00000000,8.00000000,7.09999990]]
Y=[[6.00000000,5.00000000,4.00000000]]
Result: Where(condition,X,Y)=[[9.00000000,5.00000000,7.09999990]]

## Example 2
condition=[[ True, True], [ True,False], [False, True]]
X=[[1.00000000,2.00000000], [3.00000000,4.00000000], [5.00000000,6.00000000]]
Y=[[12.00000000,11.00000000], [10.00000000, 9.00000000], [ 8.00000000, 7.00000000]]
Result: Where(condition,X,Y)=[[1.00000000,2.00000000], [3.00000000,9.00000000], [8.00000000,6.00000000]]

## Example 3a (IEEE-754)
condition=[[ True,False, True]]
X=[[19.00000000,28.00000000,37.09999847]]
Y=[[16.00000000,25.00000000,34.00000000]]
Result: Where(condition,X,Y)=[[19.00000000,25.00000000,37.09999847]]

## Example 3b (IEEE-754 special values)
condition=[[ True,False, True,False, True]]
X=[[0.00000000,0.00000000,       inf,       inf,       nan]]
Y=[[ 0.00000000,-0.00000000,       -inf,       -inf, 1.00000000]]
Result: Where(condition,X,Y)=[[ 0.00000000,-0.0000